## CHEME 5660 Lab 8: Formulation and Solution of the Linear Tiger Problem as a Markov Decision Process (MDP)

<img src="./figs/Fig-Linear-MDP-Schematic.png" style="margin:auto; width:60%"/>

## Introduction
Fill me in

## Lab 8 setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-8-MDP-Tiger-Problem`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-8-MDP-Tiger-Problem/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-8-MDP-Tiger-Problem/Manifest.toml`


In [2]:
# load req'd packages -
# ...

In [3]:
include("CHEME-5660-Lab-8-CodeLib.jl");

In [4]:
# setup some global constants -
α = 0.50; # probability of moving the direction we are expect

#### Configure states and actions

In [5]:
# setup the states and actions -
safety = 1;
tiger = 50;

states = range(safety,stop=tiger, step=1) |> collect;
actions = [1,2]; # a₁ = move left, a₂ = move right
γ = 0.95;

#### Configure the rewards array

In [6]:
# setup the rewards -
R = Array{Float64,2}(undef,length(states), length(actions));

# most of the rewards are zero -
fill!(R,0.0) # fill R w/zeros

# set the rewards for the ends -
R[safety + 1,1] = 10; # if in state 2, and we take action 1 = we live, get married, our kids are all doctors, and we are generally content
R[tiger-1, 2] = -100; # if in state N - 1, and we take action 2 = we get eaten. Bad.

#### Configure the transition array

In [7]:
# Setup the transitions
T = Array{Float64,3}(undef, length(states), length(states), length(actions));
fill!(T,0.0);

# We need to put values into the transition array (these are probabilities, so eah row much sum to 1)
T[safety, 1, 1:length(actions)] .= 1.0; # if we are in state 1, we stay in state 1 ∀a ∈ 𝒜
T[tiger, tiger, 1:length(actions)] .= 1.0; # if we are in state 5, we stay in state 5 

# left actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,1] = α;
    T[s,s+1,1] = (1-α);
end

# right actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,2] = (1-α);
    T[s,s+1,2] = α; 
end

#### Build the MDP problem object and estimate the utility $U^{\pi}(s)$ 

In [8]:
mdp_problem = build(MDPProblem; 𝒮 = states, 𝒜 = actions, T = T, R = R, γ = γ);

In [9]:
# build a always right policy -
always_move_right(s) = 2;
always_move_left(s) = 1;

In [10]:
U = iterative_policy_evaluation(mdp_problem, always_move_right, 20*length(states));

#### Estimate the Q-function

In [11]:
Q_array = Q(mdp_problem, U)[2:end-1,:]

48×2 Matrix{Float64}:
    9.99998        -1.84825e-5
   -3.89105e-5     -3.89105e-5
   -6.34344e-5     -6.34344e-5
   -9.46356e-5     -9.46356e-5
   -0.000135798    -0.000135798
   -0.000191256    -0.000191256
   -0.000266845    -0.000266845
   -0.000370524    -0.000370524
   -0.000513205    -0.000513205
   -0.000709908    -0.000709908
   -0.000981338    -0.000981338
   -0.00135607     -0.00135607
   -0.00187354     -0.00187354
    ⋮            
   -4.36306        -4.36306
   -6.02676        -6.02676
   -8.32486        -8.32486
  -11.4993        -11.4993
  -15.8841        -15.8841
  -21.941         -21.941
  -30.3074        -30.3074
  -41.8641        -41.8641
  -57.8275        -57.8275
  -79.8781        -79.8781
 -110.337        -110.337
  -52.41         -152.41

In [12]:
policy = π(Q_array)

48-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

### Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.